In [11]:
import nltk
nltk.download('all')
from nltk.tokenize import word_tokenize
from nltk import pos_tag,ne_chunk
from nltk.stem import  PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[

In [12]:
df=pd.read_csv("Womens Clothing E-Commerce Reviews.csv")

In [13]:
df=df[["Review Text","Rating"]]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [14]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
def pre_procesing(text):
    text = text.lower()
    tokens = word_tokenize(text)
    #tokens = [lemmatizer.lemmatize(word) for word in tokens]
    tokens = [stemmer.stem(lemmatizer.lemmatize(word)) for word in tokens]
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)


df["Review Text"]=df["Review Text"].apply(pre_procesing)

In [15]:
import pickle

In [16]:

with open('preprocessing_components.pkl', 'wb') as f:
  pickle.dump({'lemmatizer': lemmatizer, 'stop_words': stop_words}, f)

In [17]:
import pickle

In [18]:
df["sentiment"] = df["Rating"].apply(lambda x: "positive" if x >= 4 else "negative")

In [19]:
df["sentiment"] .value_counts()

sentiment
positive    17448
negative     5193
Name: count, dtype: int64

In [20]:
vector = TfidfVectorizer()
X= vector.fit_transform(df["Review Text"])
y=df["sentiment"]

In [21]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X,y)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, shuffle=True, stratify=y_resampled)

In [23]:

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred) )

0.8928366762177651
              precision    recall  f1-score   support

    negative       0.89      0.90      0.89      3490
    positive       0.90      0.88      0.89      3490

    accuracy                           0.89      6980
   macro avg       0.89      0.89      0.89      6980
weighted avg       0.89      0.89      0.89      6980



In [24]:
from sklearn.ensemble import RandomForestClassifier

ran = RandomForestClassifier()
ran.fit(X_train, y_train)

y_pred = ran.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred) )

0.9275071633237822
              precision    recall  f1-score   support

    negative       0.92      0.93      0.93      3490
    positive       0.93      0.92      0.93      3490

    accuracy                           0.93      6980
   macro avg       0.93      0.93      0.93      6980
weighted avg       0.93      0.93      0.93      6980



In [26]:
import joblib

In [27]:
# Save the model and vectorizer
joblib.dump(ran, 'random_forest_model_mew.pkl')
joblib.dump(vector, 'tfidf_vectorizer_new.pkl')

# Save preprocessing components in a single pickle file
with open('preprocessing_components_new.pkl', 'wb') as f:
    pickle.dump({'lemmatizer': lemmatizer, 'stop_words': stop_words}, f)